<a href="https://colab.research.google.com/github/f2010126/CodeworkWS2021/blob/main/DLCompJan15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Add Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Code from evaluate.py

In [ ]:
import os
import torch

from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
# commented for notebook
# from src.cnn import *


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt

def accuracy(logits, labels):
    preds = torch.argmax(logits, axis=1)
    return torch.sum(preds == labels) / len(labels)

def eval_fn(model, loader, device, train=False):
    """
    Evaluation method
    :param model: model to evaluate
    :param loader: data loader for either training or testing set
    :param device: torch device
    :param train: boolean to indicate if training or test set is used
    :return: accuracy on the data
    """
    score = AverageMeter()
    model.eval()

    t = tqdm(loader)
    with torch.no_grad():  # no gradient needed
        for images, labels in t:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            acc = accuracy(outputs, labels)
            score.update(acc.item(), images.size(0))

            t.set_description('(=> Test) Score: {:.4f}'.format(score.avg))

    return score.avg

def eval_model(model, saved_model_file, test_data_dir, data_augmentations):
    model = model.to(device)
    model.load_state_dict(torch.load(os.path.join(os.getcwd(), 'models', saved_model_file)))
    data = ImageFolder(test_data_dir, transform=data_augmentations)

    test_loader = DataLoader(dataset=data,
                             batch_size=128,
                             shuffle=False)

    score = eval_fn(model, test_loader, device, train=False)

    print('Avg accuracy:', str(score*100) + '%')


cnn.py


In [ ]:
""" File with CNN models. Add your custom CNN model here. """

import torch.nn as nn
import torch.nn.functional as F


class SampleModel(nn.Module):
    """
    A sample PyTorch CNN model
    """
    def __init__(self, input_shape=(3, 64, 64), num_classes=10):
        super(SampleModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=10, kernel_size=(3,3), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3,3), padding=(1,1))
        self.pool = nn.MaxPool2d(3, stride=2)
        # The input features for the linear layer depends on the size of the input to the convolutional layer
        # So if you resize your image in data augmentations, you'll have to tweak this too.
        self.fc1 = nn.Linear(in_features=4500, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# python -m src.main --model SmallCNN --epochs 25
class SmallCNN(nn.Module):
    def __init__(self, input_shape=(3, 64, 64), num_classes=10):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=6, kernel_size=3, padding=0, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(17280, 128) # <--
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

data_augmentations.py

In [ ]:
from torchvision import datasets, transforms
import torch.nn as nn

resize_to_64x64 = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

resize_and_colour_jitter = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor()
])


training.py

In [ ]:
from tqdm import tqdm
import time
import torch
import numpy as np

# commented for notebook
# from src.eval.evaluate import AverageMeter, accuracy

def train_fn(model, optimizer, criterion, loader, device, train=True):
    """
  Training method
  :param model: model to train
  :param optimizer: optimization algorithm
  :criterion: loss function
  :param loader: data loader for either training or testing set
  :param device: torch device
  :param train: boolean to indicate if training or test set is used
  :return: (accuracy, loss) on the data
  """
    time_begin = time.time()
    score = AverageMeter()
    losses = AverageMeter()
    model.train()
    time_train = 0

    t = tqdm(loader)
    for images, labels in t:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        acc = accuracy(logits, labels)
        n = images.size(0)
        losses.update(loss.item(), n)
        score.update(acc.item(), n)

        t.set_description('(=> Training) Loss: {:.4f}'.format(losses.avg))

    time_train += time.time() - time_begin
    print('training time: ' + str(time_train))
    return score.avg, losses.avg

main.py

In [ ]:
import os
import argparse
import logging
import time
import numpy as np

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchsummary import summary
from torchvision.datasets import ImageFolder

# commented for notebook
# from src.cnn import *
# from src.eval.evaluate import eval_fn, accuracy
# from src.training import train_fn
# from src.data_augmentations import *


data_dir ='/content/drive/My Drive/DEEP_LEARNING/DLCOMP/dataset'

def main(data_dir,
         torch_model,
         num_epochs=10,
         batch_size=50,
         learning_rate=0.001,
         train_criterion=torch.nn.CrossEntropyLoss,
         model_optimizer=torch.optim.Adam,
         data_augmentations=None,
         save_model_str=None,
         use_all_data_to_train=False,
         exp_name=''):
    """
    Training loop for configurableNet.
    :param data_dir: dataset path (str)
    :param num_epochs: (int)
    :param batch_size: (int)
    :param learning_rate: model optimizer learning rate (float)
    :param train_criterion: Which loss to use during training (torch.nn._Loss)
    :param model_optimizer: Which model optimizer to use during trainnig (torch.optim.Optimizer)
    :param data_augmentations: List of data augmentations to apply such as rescaling.
        (list[transformations], transforms.Composition[list[transformations]], None)
        If none only ToTensor is used
    :return:
    """

    # Device configuration
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    if data_augmentations is None:
        data_augmentations = transforms.ToTensor()
    elif isinstance(data_augmentations, list):
        data_augmentations = transforms.Compose(data_augmentations)
    elif not isinstance(data_augmentations, transforms.Compose):
        raise NotImplementedError

    # Load the dataset
    train_data = ImageFolder(os.path.join(data_dir, 'train'), transform=data_augmentations)
    val_data = ImageFolder(os.path.join(data_dir, 'val'), transform=data_augmentations)
    test_data = ImageFolder(os.path.join(data_dir, 'test'), transform=data_augmentations)

    channels, img_height, img_width = train_data[0][0].shape

    # image size
    input_shape = (channels, img_height, img_width)

    # instantiate training criterion
    train_criterion = train_criterion().to(device)
    score = []

    if use_all_data_to_train:
        train_loader = DataLoader(dataset=ConcatDataset([train_data, val_data, test_data]),
                                  batch_size=batch_size,
                                  shuffle=True)
        logging.warning('Training with all the data (train, val and test).')
    else:
        train_loader = DataLoader(dataset=train_data,
                                  batch_size=batch_size,
                                  shuffle=True)
        val_loader = DataLoader(dataset=val_data,
                                 batch_size=batch_size,
                                 shuffle=False)

    model = torch_model(input_shape=input_shape,
                        num_classes=len(train_data.classes)).to(device)

    # instantiate optimizer
    optimizer = model_optimizer(model.parameters(), lr=learning_rate)

    # Info about the model being trained
    # You can find the number of learnable parameters in the model here
    logging.info('Model being trained:')
    summary(model, input_shape,
            device='cuda' if torch.cuda.is_available() else 'cpu')

    # Train the model
    for epoch in range(num_epochs):
        logging.info('#' * 50)
        logging.info('Epoch [{}/{}]'.format(epoch + 1, num_epochs))

        train_score, train_loss = train_fn(model, optimizer, train_criterion, train_loader, device)
        logging.info('Train accuracy: %f', train_score)

        if not use_all_data_to_train:
            test_score = eval_fn(model, val_loader, device)
            logging.info('Validation accuracy: %f', test_score)
            score.append(test_score)

    if save_model_str:
        # Save the model checkpoint can be restored via "model = torch.load(save_model_str)"
        model_save_dir = os.path.join(os.getcwd(), save_model_str)

        if not os.path.exists(model_save_dir):
            os.mkdir(model_save_dir)

        save_model_str = os.path.join(model_save_dir, exp_name + '_model_' + str(int(time.time())))
        torch.save(model.state_dict(), save_model_str)

    if not use_all_data_to_train:
        logging.info('Accuracy at each epoch: ' + str(score))
        logging.info('Mean of accuracies across all epochs: ' + str(100*np.mean(score))+'%')
        logging.info('Accuracy of model at final epoch: ' + str(100*score[-1])+'%')



if __name__ == '__main__':
    """
    This is just an example of a training pipeline.

    Feel free to add or remove more arguments, change default values or hardcode parameters to use.
    """
    loss_dict = {'cross_entropy': torch.nn.CrossEntropyLoss} # Feel free to add more
    opti_dict = {'sgd': torch.optim.SGD, 'adam': torch.optim.Adam} # Feel free to add more

    cmdline_parser = argparse.ArgumentParser('DL WS20/21 Competition')

    cmdline_parser.add_argument('-m', '--model',
                                default='SmallCNN',
                                help='Class name of model to train',
                                type=str)
    cmdline_parser.add_argument('-e', '--epochs',
                                default=50,
                                help='Number of epochs',
                                type=int)
    cmdline_parser.add_argument('-b', '--batch_size',
                                default=282,
                                help='Batch size',
                                type=int)
    # cmdline_parser.add_argument('-D', '--data_dir',
    #                             default=os.path.join(os.path.dirname(os.path.abspath(__file__)),
    #                                                  '..', 'dataset'),
    #                             help='Directory in which the data is stored (can be downloaded)')
    cmdline_parser.add_argument('-l', '--learning_rate',
                                default=2.244958736283895e-05,
                                help='Optimizer learning rate',
                                type=float)
    cmdline_parser.add_argument('-L', '--training_loss',
                                default='cross_entropy',
                                help='Which loss to use during training',
                                choices=list(loss_dict.keys()),
                                type=str)
    cmdline_parser.add_argument('-o', '--optimizer',
                                default='adam',
                                help='Which optimizer to use during training',
                                choices=list(opti_dict.keys()),
                                type=str)
    cmdline_parser.add_argument('-p', '--model_path',
                                default='models',
                                help='Path to store model',
                                type=str)
    cmdline_parser.add_argument('-v', '--verbose',
                                default='INFO',
                                choices=['INFO', 'DEBUG'],
                                help='verbosity')
    cmdline_parser.add_argument('-n', '--exp_name',
                                default='default',
                                help='Name of this experiment',
                                type=str)
    cmdline_parser.add_argument('-d', '--data-augmentation',
                                default='resize_and_colour_jitter',
                                help='Data augmentation to apply to data before passing to the model.'
                                + 'Must be available in data_augmentations.py')
    cmdline_parser.add_argument('-a', '--use-all-data-to-train',
                                action='store_true',
                                help='Uses the train, validation, and test data to train the model if enabled.')

    args, unknowns = cmdline_parser.parse_known_args()
    log_lvl = logging.INFO if args.verbose == 'INFO' else logging.DEBUG
    logging.basicConfig(level=log_lvl)

    if unknowns:
        logging.warning('Found unknown arguments!')
        logging.warning(str(unknowns))
        logging.warning('These will be ignored')

    main(
        data_dir=data_dir,
        torch_model=eval(args.model),
        num_epochs=args.epochs,
        batch_size=args.batch_size,
        learning_rate=args.learning_rate,
        train_criterion=loss_dict[args.training_loss],
        model_optimizer=opti_dict[args.optimizer],
        data_augmentations=eval(args.data_augmentation),  # Check data_augmentations.py for sample augmentations
        save_model_str=args.model_path,
        exp_name=args.exp_name,
        use_all_data_to_train=args.use_all_data_to_train
    )


INFO:root:Model being trained:
INFO:root:##################################################
INFO:root:Epoch [1/50]
  0%|          | 0/2 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 62, 62]             168
         MaxPool2d-2            [-1, 6, 31, 31]               0
            Conv2d-3           [-1, 16, 29, 29]             880
         MaxPool2d-4           [-1, 16, 14, 14]               0
            Conv2d-5          [-1, 120, 12, 12]          17,400
            Linear-6                  [-1, 128]       2,211,968
            Linear-7                   [-1, 10]           1,290
Total params: 2,231,706
Trainable params: 2,231,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.48
Params size (MB): 8.51
Estimated Total Size (MB): 9.04
----------------------------------------------------------------


(=> Training) Loss: 2.3038: 100%|██████████| 2/2 [01:53<00:00, 56.52s/it]
INFO:root:Train accuracy: 0.100000
  0%|          | 0/1 [00:00<?, ?it/s]

training time: 113.03406548500061


(=> Test) Score: 0.1100: 100%|██████████| 1/1 [00:54<00:00, 54.61s/it]
INFO:root:Validation accuracy: 0.110000
INFO:root:##################################################
INFO:root:Epoch [2/50]
(=> Training) Loss: 2.3000: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]
INFO:root:Train accuracy: 0.105000
  0%|          | 0/1 [00:00<?, ?it/s]

...

(=> Test) Score: 0.3950: 100%|██████████| 1/1 [00:01<00:00,  1.42s/it]
INFO:root:Validation accuracy: 0.395000
INFO:root:##################################################
INFO:root:Epoch [49/50]
(=> Training) Loss: 2.0670: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]
INFO:root:Train accuracy: 0.425000
  0%|          | 0/1 [00:00<?, ?it/s]

training time: 2.8492393493652344


(=> Test) Score: 0.4050: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]
INFO:root:Validation accuracy: 0.405000
INFO:root:##################################################
INFO:root:Epoch [50/50]
(=> Training) Loss: 2.0592: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]
INFO:root:Train accuracy: 0.430000
  0%|          | 0/1 [00:00<?, ?it/s]

training time: 2.850365400314331


(=> Test) Score: 0.4450: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]
INFO:root:Validation accuracy: 0.445000
INFO:root:Accuracy at each epoch: [0.10999999940395355, 0.10499999672174454, 0.10999999940395355, 0.11499999463558197, 0.14000000059604645, 0.14499999582767487, 0.17000000178813934, 0.1599999964237213, 0.19999998807907104, 0.20499999821186066, 0.19499999284744263, 0.1899999976158142, 0.2149999886751175, 0.22999998927116394, 0.2199999988079071, 0.2149999886751175, 0.23499999940395355, 0.25999999046325684, 0.25, 0.2800000011920929, 0.29499998688697815, 0.29999998211860657, 0.32499998807907104, 0.35499998927116394, 0.35999998450279236, 0.35499998927116394, 0.3999999761581421, 0.39499998092651367, 0.38999998569488525, 0.4050000011920929, 0.39499998092651367, 0.38999998569488525, 0.4099999964237213, 0.3799999952316284, 0.38999998569488525, 0.3999999761581421, 0.3799999952316284, 0.4050000011920929, 0.38999998569488525, 0.4099999964237213, 0.41499999165534973, 0.41999998688697815, 0